In [21]:
from clickhouse_driver import Client
import pandas as pd
import plotly.express as px

In [ ]:
# Connect to ClickHouse
try:
	client = Client(host='localhost', port=9005, user='default', password='', database='marts')
	# Read the cluster analysis table
	query = 'SELECT * FROM customers_clusters_analysis'
	result = client.execute(query)

	# Convert result to a pandas DataFrame for easier analysis (optional)
	df = pd.DataFrame(result, columns=[desc[0] for desc in client.execute('DESCRIBE TABLE customers_clusters_analysis')])

	display(df.head())
except Exception as e:
	print("Error connecting to ClickHouse:", e)

In [17]:
# ------------------------------
# 1. Calculate diffs between each consecutive timestamp
# ------------------------------
def calculate_all_diffs(df, metrics):
    diffs = []
    for cluster_id in df['cluster'].unique():
        cluster_df = df[df['cluster'] == cluster_id].sort_values('analysis_timestamp')
        for i in range(1, len(cluster_df)):
            prev = cluster_df.iloc[i-1]
            curr = cluster_df.iloc[i]
            diff = {metric: curr[metric] - prev[metric] for metric in metrics}
            diff['cluster'] = cluster_id
            diff['analysis_timestamp'] = curr['analysis_timestamp']  # timestamp where change happened
            diffs.append(diff)
    return pd.DataFrame(diffs)

metrics = [col for col in df.columns if col not in ['cluster', 'analysis_timestamp']]
diff_df = calculate_all_diffs(df, metrics)

# ------------------------------
# 2. Plot line charts with diffs
# ------------------------------
for metric in metrics:
    fig = px.line(
        df,
        x="analysis_timestamp",
        y=metric,
        color="cluster",
        markers=True,
        title=f"{metric} trend over time by cluster"
    )
    
    # Add annotations for ALL differences
    for _, row in diff_df.iterrows():
        cluster_id = row['cluster']
        diff_val = row[metric]
        ts = row['analysis_timestamp']
        val = df[(df['cluster'] == cluster_id) & (df['analysis_timestamp'] == ts)][metric].values[0]
        
        fig.add_annotation(
            x=ts,
            y=val,
            text=f"Δ {diff_val:.5f}",
            showarrow=True,
            arrowhead=2,
            font=dict(color="red" if diff_val < 0 else "green")
        )
    
    fig.show()


In [ ]:
# generate cluster insights for all clusters
def generate_cluster_insights(diff_df):
    insights = []
    
    for _, row in diff_df.iterrows():
        cluster_id = row["cluster"]

        # Population change
        pop_change = row["rows_in_cluster"]
        if pop_change > 0:
            pop_text = f"Cluster {cluster_id} grew by {pop_change:,.0f} members."
        elif pop_change < 0:
            pop_text = f"Cluster {cluster_id} shrank by {abs(pop_change):,.0f} members."
        else:
            pop_text = f"Cluster {cluster_id} population remained stable."
        
        # Spend changes
        spend_change = row["avg_total_spend_usd"]
        order_value_change = row["avg_avg_order_value_usd"]
        if spend_change > 0:
            spend_text = f"Average spend increased by ${spend_change:.5f} (AOV change: ${order_value_change:.5f})."
        elif spend_change < 0:
            spend_text = f"Average spend decreased by ${abs(spend_change):.5f} (AOV change: ${order_value_change:.5f})."
        else:
            spend_text = "Average spend remained unchanged."

        # Recency
        recency_change = row["avg_days_since_last_purchase"]
        if recency_change < 0:
            recency_text = f"Customers purchased more recently ({recency_change:.5f} days sooner)."
        elif recency_change > 0:
            recency_text = f"Customers purchased less recently ({recency_change:.5f} days later)."
        else:
            recency_text = "Purchase recency unchanged."

        # Web activity
        web_change = row["avg_days_since_last_web_activity"]
        if web_change < 0:
            web_text = f"Web activity improved ({web_change:.5f} days sooner)."
        elif web_change > 0:
            web_text = f"Web activity declined ({web_change:.5f} days later)."
        else:
            web_text = "Web activity unchanged."

        # Combine
        insight = f"""
        📊 **Cluster {cluster_id} Insights**  
        - {pop_text}  
        - {spend_text}  
        - {recency_text}  
        - {web_text}  
        """
        insights.append(insight)
    
    return "\n".join(insights)


# Generate insights
print(generate_cluster_insights(diff_df))


        📊 **Cluster 0 Insights**  
        - Cluster 0 shrank by 183,424 members.  
        - Average spend increased by $0.00140 (AOV change: $0.00140).  
        - Customers purchased more recently (-0.05828 days sooner).  
        - Web activity improved (-0.60100 days sooner).  
        

        📊 **Cluster 1 Insights**  
        - Cluster 1 grew by 129,126 members.  
        - Average spend increased by $10.08858 (AOV change: $10.09327).  
        - Customers purchased more recently (-0.74281 days sooner).  
        - Web activity improved (-0.93306 days sooner).  
        

        📊 **Cluster 2 Insights**  
        - Cluster 2 grew by 54,298 members.  
        - Average spend increased by $4.87550 (AOV change: $6.65176).  
        - Customers purchased more recently (-0.47544 days sooner).  
        - Web activity improved (-0.87682 days sooner).  
        
